# Chapter 19: Concurrency Models in Python

> Concurrency is about dealing with lots of things at once.
> 
> Parallelism is about doing lots of things at once.
> 
> Not the same, but related.
> 
> One is about structure, one is about execution.
> 
> Concurrency provides a way to structure a solution to solve a problem that may (but not necessarily) be parallelizable.
> 
> -- Rob Pike, co-creator of Go

## A Bit of Jargon

- Concurrency: The ability to handle multiple pending tasks, making progress one at a time or in parallel(if possible) so that each of them eventually succeeds or fails. A single-core CPU is capable of concurrency if it runs an OS scheduler that interleaves the execution of different tasks. Also known as _multi-tasking_.

- Parallelism: The ability to run multiple tasks _at the same time_. This requires a multi-core CPU, multiple CPUs, a GPU, or a cluster of computers. Parallelism is about _speeding up_ computation by using extra computing resources.

- Execution unit: General term for objects that execute code concurrently, each with independent state and call stack. Python natively supports three kinds of execution units: processes, threads, and coroutines.

- Process: An instance of a computer program while it is running, using memory and a slice of the CPU time. Modern desktop operating systems routinely manage hundreds of processes concurrently, with each process isolated in its own private memory space. Processes communicate via pipes, sockets, or memory mapped files -- all of which can only carry raw bytes. Python objects must be serialized(converted) into raw bytes to pass from one process to another. This is costly, and not all Python objects are serializable. A process can spawn subprocesses, each called a child process. These are also isolated from each other and from the parent. Processes allow _preemptive multitasking_: the OS scheduler _preempts_ --i.e., suspends -- each running process periodically to allow other processes to run. This means that a frozen process can't freeze the whole system -- in theory.

- Thread: An execution unit within a single process. When a process starts, it uses a single thread: the _main thread_. A process can create more threads to operate concurrently by calling operating system APIs. Threads within a process share the same memory space, which holds live Python objects. This allows easy data sharing between threads, but can also lead to corrupted data when more than one thread updates the same object concurrently. Like processes, threads also enable _preemptive multitasking_ under the supervision of the OS scheduler. A thread consumes less resources than a process doing the same job.

- Coroutine: A function that can suspend itself and resume later. In Python, _classic coroutines_ are built from generator functions, and _native coroutines_ are defined with _async def_. "Classic Coroutines" on page 641 introduced the concept, and Chapter 21 covers the use of the supervision of an _event loop_, also in the same thread. Asynchronous programming frameworks such as _asyncio_, _Curio_, or _Trio_ provide an event loop and supporting libraries for nonblocking, coroutine-based I/O. Coroutines support _cooperative multitasking_: each coroutine must explicitly cede control with the _yield_ or _await_ keywords, so that another may proceed concurrently(but not in parallel). This means that any blocking code in coroutine blocks the execution of the event loop and all other coroutines--in contrast with the _preemptive multitasking_ of processes and threads. On the other hand, each coroutine consumes less resources that a thread doing the same job.

- Queue: A data structure that lets us put and get items, usually in FIFO order: first in, first out. Queues allow separate execution units to exchange application data and control messages, such as error codes and signals to terminate. The implementation of a queue varies according to the underlying concurrency model: the _queue_ package in Python's standard library provides queue classes to support threads, while the _multiprocessing_ and _asyncio_ packages implement their own queue classes. The _queue_ and _asyncio_ packages also include queues that are not FIFO: _PriorityQueue_ and _LifoQueue_.

- Lock: An object that execution units can use to synchronize their actions and avoid corrupting data. While updating a shared data structure, the running code should hold an associated lock. This signals other parts of the program to wait until the lock is released before accessing the same structure. The simplest type of lock is also known as a mutex(for mutual exclusion). The implementation of a lock depends on the underlying concurrency model.

- Contention: Dispute over a limited asset. Resource contention happens when multiple execution units try to access a shared resource--such as a lock or storage. There's also CPU contention, when compute-intensive processes or threads must wait for the OS scheduler to give them CPU time.

## Processes, Threads, and Python's infamous GIL

Here is how concepts we just saw apply to Python programming, in 10 points:

1. Each instance of the Python interpreter is a process. You can start additional Python processes using `multiprocessing` or `concurrent.futures` libraries. Python `subprocess` library is designed to launch processes to run external programs, regardless of the languages used to write them.

2. The Python interpreter uses a single thread to run the user's program and the memory garbage collector. You can start additional Python threads using the `threading` or `concurrent.futures` libraries.

3. Access to object reference counts and other internal interpreter state is controlled by a lock, the Global Interpreter Lock(GIL). Only one Python thread can hold the GIL at any time. This means that only one thread can execute Python code at any time, regardless of the number of CPU cores.

4. To prevent a Python thread from holding the GIL indefinitely, Python's bytecode interpreter pauses the current Python thread every 5ms by default, releasing the GIL. The thread can then try to reacquire the GIL, but if there are other threads waiting for it, the OS scheduler may pick one of them to proceed.

In [11]:
import sys

# Call sys.getswitchinterval() to get the current value of the switch interval
print(sys.getswitchinterval())

# Change it with sys.setswitchinterval()
sys.setswitchinterval(0.008)
print(sys.getswitchinterval())
sys.setswitchinterval(0.005)

0.005
0.008
